# ML projet - Groupe 3

# Pretraitement

In [100]:
import pandas as pd
from gensim.models import Word2Vec
import nltk
import re
from nltk import word_tokenize, pos_tag
from nltk.corpus import words
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
import enchant
import numpy as np
from collections import Counter
import seaborn as sns


from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
import random
from sklearn.model_selection import KFold



from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import cross_val_score

from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

import emoji
from scipy.sparse import hstack
from sklearn.linear_model import LogisticRegression
from nltk.corpus import stopwords



#nltk.download('punkt_tab')
#nltk.download("words")
#nltk.download('wordnet')
#nltk.download('averaged_perceptron_tagger_eng')
#%matplotlib qt
#nltk.download('stopwords')
%matplotlib inline

In [101]:
df = pd.read_csv("scitweets_export.tsv", sep = "\t")

In [102]:
len(df[df["science_related"] == 0])/(len(df))

0.6710526315789473

### Enlever (.,\\@#:';"“”’[]), les chiffres et les liens (http) de chaque tweet

In [103]:
text = []
all_text = list(df["text"])
for t in all_text:
    a = re.sub("[\.,/\\@:;\"\'0-9“”’\[\]]", "", t)
    #a = re.sub("http\S*", "", a) #Enlever les liens
    a = re.sub("#\S*", "", a) #Enlever les tags
    a = a.lower()
    text.append(a)
#for sentence in text:  

In [104]:
#Créer une liste des phrases ou chaque phrase est une liste de mots.
text_sep = [word_tokenize(t) for t in text]

In [105]:
#len(text_sep)
text_sep[-3]

['whats', 'the', 'uber', 'support', 'team', 'email', 'address', '?']

## Standardizer chaque phrases. 
- Convertir les noms pluriels en leur forme singuliers
- Convertir les conjugaison de chaque verbe en leur infinitif
- Standardiser les ajectives et les adverbes

In [106]:
def get_wordnet_pos(tag):
    if tag.startswith('J'):  # Adjectif
        return wordnet.ADJ
    elif tag.startswith('V'):  # Verbe
        return wordnet.VERB
    elif tag.startswith('N'):  # Nom
        return wordnet.NOUN
    elif tag.startswith('R'):  # Adverbe
        return wordnet.ADV
    else:
        return wordnet.NOUN 

    
lemmatizer = WordNetLemmatizer()
refined_tweets = []
for tweet in text_sep:
    pos_tags = pos_tag(tweet)
    lemmatized_words = [lemmatizer.lemmatize(word, get_wordnet_pos(tag)) for word, tag in pos_tags]
    refined_tweets.append(lemmatized_words)



In [111]:
tfidf = TfidfVectorizer(max_features=100000)  # tu choisis max_features selon ton cas
X_text = tfidf.fit_transform(remake_sentences((refined_tweets)))
y = df["science_related"]

kf = KFold(n_splits=10, shuffle=True, random_state=42)

In [112]:
logreg = LogisticRegression(class_weight='balanced', max_iter=1000)

# Faire 5-fold cross-validation
scores_rf = cross_val_score(logreg, X_text, y, cv=kf, scoring='accuracy')

print("Scores par fold (logisitic regression):", scores_rf)
print("Score moyen:", scores_rf.mean())

Scores par fold (logisitic regression): [0.78947368 0.77192982 0.76315789 0.75438596 0.85087719 0.74561404
 0.86842105 0.81578947 0.81578947 0.85087719]
Score moyen: 0.8026315789473684


## Laisser une trace pour les emojis

In [109]:
emoji_list = []
has_emoji = False 
for sentence in refined_tweets:
    has_emoji = False
    for word in sentence:
        for char in word:
            if emoji.is_emoji(char):
                has_emoji = True
                break
    if has_emoji is True:
        emoji_list.append(1)
    else:
        emoji_list.append(0)
df["has_emoji"] = emoji_list

In [68]:
def remake_sentences(sentence_list):
    sentence_join = []
    phrase = ""
    for sentence in sentence_list:
        for word in sentence:
            phrase = phrase + word + " "
        sentence_join.append(phrase)
        phrase = ""
    return(sentence_join)

In [69]:
def plot_word_distribution(word_list):
    word_list_flat = [word for sublist in word_list for word in sublist]
    word_counts = Counter(word_list_flat)
    # 3. Afficher les 10 mots les plus fréquents (optionnel)
    most_common = word_counts.most_common(100)
    #print(most_common[0:10])
    words, counts = zip(*most_common)
    
    # 4. Tracer avec seaborn
    #plt.figure(figsize=(30, 12))
    plt.figure(figsize=(16, 8)).gca()
    sns.barplot(x=list(words), y=list(counts))
    plt.title("Distribution des mots")
    plt.ylabel("Nombre d'occurrences")
    plt.xlabel("Mots")
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()


In [54]:
plot_word_distribution(refined_tweets)

/home/cm/.local/lib/python3.8/site-packages/matplotlib/backends/backend_agg.py:238: RuntimeWarning: Glyph 1 missing from current font.
  font.set_text(s, 0.0, flags=flags)


In [55]:
mots_a_supprimer = ["the", "to", "a", "of", "and", "in", "for", "on", "this", 
                    "with", "that", "have", "by", "it", "from", "be", "do", 
                    "t", " ", "to", "\x01", "co", "no", "out", "d", "our", "so",
                   "(", ")", "your", "but", "if", "than", "\x01"]

# Supprimer toutes les occurrences de "the"
filtered_words = [[word for word in row if word not in mots_a_supprimer] for row in refined_tweets]
#filtered_words = re.sub(r"[\x00-\x1F]", filtered_words)

In [56]:
plot_word_distribution(filtered_words)

In [57]:
X = remake_sentences(filtered_words)  # Utiliser uniquement la colonne 'text' comme feature
y_binary = df['science_related']  # Étiquette binaire (scientifique ou non)


X_train, X_test, y_train, y_test = train_test_split(X, y_binary, test_size=0.2, random_state=1)

vectorizer = TfidfVectorizer(ngram_range=(1, 2), max_features=100000)
X_vec = vectorizer.fit_transform(X)  # Apprentissage et transformation sur l'entraînement

X_train_vec = vectorizer.fit_transform(X_train)  # Apprentissage et transformation sur l'entraînement


X_test_vec = vectorizer.transform(X_test)

/home/cm/.local/lib/python3.8/site-packages/matplotlib/backends/backend_agg.py:238: RuntimeWarning: Glyph 1 missing from current font.
  font.set_text(s, 0.0, flags=flags)
/home/cm/.local/lib/python3.8/site-packages/matplotlib/backends/backend_agg.py:201: RuntimeWarning: Glyph 1 missing from current font.
  font.set_text(s, 0, flags=flags)


In [58]:
len(X_test_vec.data)

2128

In [41]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline


X_train, X_test, y_train, y_test = train_test_split(X, y_binary, test_size=0.2, random_state=42)

# Création du pipeline
pipeline = ImbPipeline([
    ('vectorizer', TfidfVectorizer()),
    ('smote', SMOTE(random_state=42)),
    ('classifier', RandomForestClassifier(class_weight='balanced', random_state=42))
])

# Entraînement
pipeline.fit(X_train, y_train)

# Prédiction
y_pred = pipeline.predict(X_test)

# Évaluation
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[139   7]
 [ 53  29]]
              precision    recall  f1-score   support

           0       0.72      0.95      0.82       146
           1       0.81      0.35      0.49        82

    accuracy                           0.74       228
   macro avg       0.76      0.65      0.66       228
weighted avg       0.75      0.74      0.70       228



In [42]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline


#X_train, X_test, y_train, y_test = train_test_split(list(df["text"]), list(df["science_related"]), test_size=0.2, random_state=42)

# Création du pipeline
pipeline = ImbPipeline([
    ('vectorizer', TfidfVectorizer()),
    ('smote', SMOTE(random_state=42)),
    ('classifier', RandomForestClassifier(class_weight='balanced', random_state=42))
])

scores = cross_val_score(pipeline, list(df["text"]), list(df["science_related"]), cv=5, scoring='f1')
print("F1 scores:", scores)
print("Moyenne:", scores.mean())

F1 scores: [0.31914894 0.60504202 0.4957265  0.70072993 0.63865546]
Moyenne: 0.5518605675791208


In [35]:
from sklearn.svm import SVC
from imblearn.pipeline import Pipeline as ImbPipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from imblearn.over_sampling import SMOTE
import pandas as pd


X_train, X_test, y_train, y_test = train_test_split(list(df["text"]), list(df["science_related"]), test_size=0.2, random_state=42)

# Pipeline avec SVM
pipeline_svm = ImbPipeline([
    ('vectorizer', TfidfVectorizer()),
    ('smote', SMOTE(random_state=42)),
    ('classifier', SVC(class_weight='balanced', kernel='linear', random_state=42))
])

# Entraînement
pipeline_svm.fit(X_train, y_train)

# Prédiction
y_pred = pipeline_svm.predict(X_test)

# Évaluation
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))


[[132  14]
 [ 25  57]]
              precision    recall  f1-score   support

           0       0.84      0.90      0.87       146
           1       0.80      0.70      0.75        82

    accuracy                           0.83       228
   macro avg       0.82      0.80      0.81       228
weighted avg       0.83      0.83      0.83       228



In [36]:
# Réduire la dimension à 2D avec PCA
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_vec.toarray())  # Convertir la matrice creuse en tableau dense

# Visualiser les données en 2D
plt.figure(figsize=(10, 6))
plt.scatter(X_pca[:, 0], X_pca[:, 1], c=y_binary, cmap='viridis', alpha=0.6)
plt.colorbar(label='science_related')
plt.title('Visualisation des tweets vectorisés (PCA 2D)')
plt.xlabel('Composante principale 1')
plt.ylabel('Composante principale 2')

plt.axvline(x=-0.06, color='red', linestyle='--', linewidth=2)  
plt.axvline(x=0, color='red', linestyle='--', linewidth=2)  

plt.axhline(y=0.07, color='red', linestyle='--')
plt.axhline(y=-0.05, color='red', linestyle='--')

plt.show()

/home/cm/.local/lib/python3.8/site-packages/matplotlib/backends/backend_agg.py:238: RuntimeWarning: Glyph 1 missing from current font.
  font.set_text(s, 0.0, flags=flags)
/home/cm/.local/lib/python3.8/site-packages/matplotlib/backends/backend_agg.py:201: RuntimeWarning: Glyph 1 missing from current font.
  font.set_text(s, 0, flags=flags)


In [156]:
len(refined_tweets)

1140

In [157]:
pca_df = pd.DataFrame({
    'pca1': X_pca[:, 0],
    'pca2': X_pca[:, 1],
    'label': y_binary,
    'tweet': filtered_words  # doit être dans le même ordre que X_train
})

In [105]:
print(pca_df['pca1'].min(), pca_df['pca1'].max())
print(pca_df['pca2'].min(), pca_df['pca2'].max())

-0.08623697698364022 0.2517037492972924
-0.14729107119292217 0.7191086346164846


In [106]:
zone = pca_df[
    (pca_df['pca2'] > -0.05) & (pca_df['pca2'] < 0.07) &  # limites vertical
    (pca_df['pca1'] > -0.06) & (pca_df['pca1'] < 0.0)    # limites horizontal
]

In [108]:
print(len(zone[zone["label"] == 0]))
print(len(zone[zone["label"] == 1]))

446
303


In [109]:
zone.to_csv("pca_zone.csv", index=False)

In [110]:
df[df["science_related"] == 1].to_csv("science_related.csv", index=False)

In [303]:
# Réduire la dimension à 2D avec t-SNE
tsne = TSNE(n_components=3, random_state=42, perplexity=15)
X_train_tsne = tsne.fit_transform(X_train_vec.toarray())

# Visualiser les données en 2D
plt.figure(figsize=(10, 6))
plt.scatter(X_train_tsne[:, 0], X_train_tsne[:, 1], c=y_train, cmap='viridis', alpha=0.6)
plt.colorbar(label='science_related')
plt.title('Visualisation des tweets vectorisés (t-SNE 2D)')
plt.xlabel('Composante t-SNE 1')
plt.ylabel('Composante t-SNE 2')
plt.show()